In [1]:
from init import *

In [7]:
from pyeoskit import db
db.reset()

code = r'''
#include <eosio/types.h>
#include <eosio/action.h>
#include <eosio/print.h>
#include <eosio/name.hpp>

#define N(name) #name##_n.value

extern "C" {

__attribute__((eosio_wasm_import))
int call_contract_get_extra_args(void* extra_args, size_t size1);

__attribute__((eosio_wasm_import))
int call_contract_set_results(void* result, size_t size1);

    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        uint64_t n = 0;
        float f = 10.8;
        for (int i=0;i<100;i++) {
            n += i;
            f += (float)i;
        }
        prints("goodbye,world\n");
    }
    
    __attribute__((eosio_wasm_entry))
    void call(uint64_t func, uint64_t arg1, uint64_t arg2, uint64_t arg3) {
      if (func == N(calltest1)) {
         uint64_t extra_args;
         call_contract_get_extra_args(&extra_args, sizeof(extra_args));
         extra_args += 1;
         call_contract_set_results(&extra_args, sizeof(extra_args));
      }
   }
}
'''


r = open('test.cpp', 'w').write(code)

if  compile_cpp('test'):
    account_name = 'helloworld11'
    code = open('test.wasm', 'rb').read()
    m = hashlib.sha256()
    m.update(code)
    code_hash = m.hexdigest()

    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        print('update contract')
        abi = ''
        r = eosapi.set_contract(account_name, code, abi, 0)
print('done!')


clang_7 output: 
wasm_ld output: 
update contract
done!


In [329]:
from pyeoskit import db
db.reset()
code = '''
def apply(receiver, code, action):
    ret = call_contract('helloworld33', 'calltest1', 0, 0, 0, int.to_bytes(1, 8, 'little'))
    print(ret)
    print(int.from_bytes(ret, 'little'))
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

True


279
